In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "OpenLLM-Ro/RoMistral-7b-Instruct-DPO"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
import json
import torch

SYSTEM_MSG = """
Ești un evaluator specializat în documente juridice redactate în limba română.

Analizezi o cerere juridică generată anterior și evaluezi următoarele criterii:

1. Respectarea structurii documentului (titlu, preambul, corp, părți implicate). — Notă 1-10
2. Limbajul formal. — Notă 1-10
3. Respectarea scopului juridic declarat (cerere, recurs, plângere etc.). — Notă 1-10
4. Observații: probleme, lipsuri, incoerențe.

Returnezi STRICT acest format JSON:

{
  "structura": { "nota": <1-10> },
  "limbaj_formal": { "nota": <1-10> },
  "respectarea_scopului": { "nota": <1-10> },
  "observatii_generale": "<text>"
}

Nu adăuga text în afara acestui JSON.
"""

TEMPLATE = """
Analizează următoarea cerere juridică:

{document_text}

Respectă instrucțiunile din SYSTEM și returnează strict obiectul JSON cerut.
"""

# ==========================================
# 1. GENERARE PROMPT OPTIMIZATĂ
# ==========================================
def generate_evaluation_prompt(cerere):
    # Limităm inputul enorm → evităm OOM
    cerere_text = json.dumps(cerere, ensure_ascii=False, indent=2)
    if len(cerere_text) > 15000:
        cerere_text = cerere_text[:15000] + "\n... [trunchiat]"

    prompt = TEMPLATE.format(document_text=cerere_text)
    return prompt


# ==========================================
# 2. APEL MODEL OPTIMIZAT (ANTI-OOM)
# ==========================================
def call_model(system, prompt, meta, temperature=0.0, max_new_tokens=300):

    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"

    # Tokenizer pe CPU -> reduce memorie GPU
    inputs = tokenizer(full_prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=False,  # determinist, consum mic
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # extrage doar răspunsul generat
    generated_only = decoded[len(full_prompt):].strip()
    generated_only = generated_only.replace("\\n", "\n")

    print("\n--- Text generat ---")
    print(generated_only)
    print("-------------------\n")

    data = {
        "Cerere": meta,
        "Evaluare": generated_only
    }

    # salvare JSON
    try:
        with open("evaluare_cereri.json", "r", encoding="utf-8") as f:
            existing = json.load(f)
            if not isinstance(existing, list):
                existing = []
    except FileNotFoundError:
        existing = []

    existing.append(data)

    with open("evaluare_cereri.json", "w", encoding="utf-8") as f:
        json.dump(existing, f, ensure_ascii=False, indent=4)

    # eliberăm memorie GPU
    del outputs, inputs
    torch.cuda.empty_cache()

    return generated_only


# ==========================================
# 3. EXECUȚIE COMPLETĂ OPTIMIZATĂ
# ==========================================
with open("/content/istoric_cereri .json", "r", encoding="utf-8") as f:
    cereri = json.load(f)

print(f"Au fost găsite {len(cereri)} cereri în istoric.\n")

to_be_evaluated_requests = cereri[:60]

for idx, cerere in enumerate(to_be_evaluated_requests):
    print(f"\n=== Analizez cererea {idx + 1} / {len(to_be_evaluated_requests)} ===")

    prompt = generate_evaluation_prompt(cerere)
    rezultat = call_model(SYSTEM_MSG, prompt, cerere)

    print("\nRăspuns model (evaluare):")

print("\nEvaluările au fost salvate în evaluare_cereri.json")


Au fost găsite 120 cereri în istoric.


=== Analizez cererea 1 / 60 ===

--- Text generat ---
{
  "structura": { "nota": 9 },
  "limbaj_formal": { "nota": 9 },
  "respectarea_scopului": { "nota": 9 },
  "observatii_generale": ""
}

Documentul respectă structura tipică a unei cereri de recurs, cu un titlu, preambul, corp și părți implicate. Limbajul este formal, iar documentul îndeplinește scopul declarat de a contesta o hotărâre pronunțată de o instanță și de a solicita o soluționare a unui conflict juridic. Nu sunt observate probleme, lipsuri sau incoerențe.
-------------------


Răspuns model (evaluare):

=== Analizez cererea 2 / 60 ===

--- Text generat ---
{
  "structura": { "nota": 10 },
  "limbaj_formal": { "nota": 10 },
  "respectarea_scopului": { "nota": 10 },
  "observatii_generale": ""
}
-------------------


Răspuns model (evaluare):

=== Analizez cererea 3 / 60 ===

--- Text generat ---
{
  "structura": { "nota": 10 },
  "limbaj_formal": { "nota": 10 },
  "respectarea_scopu

In [ ]:
import json
import torch

SYSTEM_MSG = """
Ești un evaluator specializat în documente juridice redactate în limba română.

Analizezi o cerere juridică generată anterior și evaluezi următoarele criterii:

1. Respectarea structurii documentului (titlu, preambul, corp, părți implicate). — Notă 1-10
2. Limbajul formal. — Notă 1-10
3. Respectarea scopului juridic declarat (cerere, recurs, plângere etc.). — Notă 1-10
4. Observații: probleme, lipsuri, incoerențe.

Returnezi STRICT acest format JSON:

{
  "structura": { "nota": <1-10> },
  "limbaj_formal": { "nota": <1-10> },
  "respectarea_scopului": { "nota": <1-10> },
  "observatii_generale": {<text>}
}

Nu adăuga text în afara acestui JSON.
"""

TEMPLATE = """
Analizează următoarea cerere juridică:

{document_text}

Respectă instrucțiunile din SYSTEM și returnează strict obiectul JSON cerut.
"""

# ==========================================
# 1. GENERARE PROMPT OPTIMIZATĂ
# ==========================================
def generate_evaluation_prompt(cerere):
    # Limităm inputul enorm → evităm OOM
    cerere_text = json.dumps(cerere, ensure_ascii=False, indent=2)
    if len(cerere_text) > 15000:
        cerere_text = cerere_text[:15000] + "\n... [trunchiat]"

    prompt = TEMPLATE.format(document_text=cerere_text)
    return prompt


# ==========================================
# 2. APEL MODEL OPTIMIZAT (ANTI-OOM)
# ==========================================
def call_model(system, prompt, meta, temperature=0.0, max_new_tokens=300):

    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"

    # Tokenizer pe CPU -> reduce memorie GPU
    inputs = tokenizer(full_prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=False,  # determinist, consum mic
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # extrage doar răspunsul generat
    generated_only = decoded[len(full_prompt):].strip()
    generated_only = generated_only.replace("\\n", "\n")

    print("\n--- Text generat ---")
    print(generated_only)
    print("-------------------\n")

    data = {
        "Cerere": meta,
        "Evaluare": generated_only
    }

    # salvare JSON
    try:
        with open("evaluare_cereri.json", "r", encoding="utf-8") as f:
            existing = json.load(f)
            if not isinstance(existing, list):
                existing = []
    except FileNotFoundError:
        existing = []

    existing.append(data)

    with open("evaluare_cereri.json", "w", encoding="utf-8") as f:
        json.dump(existing, f, ensure_ascii=False, indent=4)

    # eliberăm memorie GPU
    del outputs, inputs
    torch.cuda.empty_cache()

    return generated_only


# ==========================================
# 3. EXECUȚIE COMPLETĂ OPTIMIZATĂ
# ==========================================
with open("/content/istoric_cereri .json", "r", encoding="utf-8") as f:
    cereri = json.load(f)

print(f"Au fost găsite {len(cereri)} cereri în istoric.\n")

to_be_evaluated_requests = cereri[60:]

for idx, cerere in enumerate(to_be_evaluated_requests):
    print(f"\n=== Analizez cererea {idx + 1} / {len(to_be_evaluated_requests)} ===")

    prompt = generate_evaluation_prompt(cerere)
    rezultat = call_model(SYSTEM_MSG, prompt, cerere)

    print("\nRăspuns model (evaluare):")

print("\nEvaluările au fost salvate în evaluare_cereri.json")


Au fost găsite 120 cereri în istoric.


=== Analizez cererea 1 / 60 ===

--- Text generat ---
{
  "structura": { "nota": 9 },
  "limbaj_formal": { "nota": 9 },
  "respectarea_scopului": { "nota": 9 },
  "observatii_generale": ""
}
-------------------


Răspuns model (evaluare):

=== Analizez cererea 2 / 60 ===

--- Text generat ---
{
  "structura": {
    "nota": 10
  },
  "limbaj_formal": {
    "nota": 10
  },
  "respectarea_scopului": {
    "nota": 10
  },
  "observatii_generale": ""
}
-------------------


Răspuns model (evaluare):

=== Analizez cererea 3 / 60 ===

--- Text generat ---
{
  "structura": {
    "nota": 10
  },
  "limbaj_formal": {
    "nota": 10
  },
  "respectarea_scopului": {
    "nota": 10
  },
  "observatii_generale": "Nicio problemă, lipsă sau incoerență observată în document. Cererea juridică respectă structura, limbajul formal și scopul declarat."
}
-------------------


Răspuns model (evaluare):

=== Analizez cererea 4 / 60 ===

--- Text generat ---
{
  "structu